# Contrôle des modèles décisionnels

Ce document a pour objectif de fournir des fichiers servant de base au contrôle de la complétude des modèles décisionnels (MD). 

Comme indiqué ci-dessous, plusieurs informations seront compilées à différentes échelles : 
- le nombre de cultures déclarées dans les leviers du MD
- le nombre de leviers total déclarés dans les rubriques du MD
- le nombre de catégories d'objectifs dans les rubriques du MD
- le nombre de levier par type de rubrique dans le MD

<img src="images/modele_decisionnel_controle.png" alt="drawing" width="600"/>

ATTENTION : pensez bien à mettre à jour les arguments de filtre chaque annee

In [406]:
import pandas as pd
from tqdm import tqdm
import numpy as np

/home/bvuittenez/anaconda3/envs/agrosyst_entrepot/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [407]:
# Campagne de controle, le controle sera effectue sur cette campagne et sur n-1
campagne_controle = 2024

#nom du fichier des attendus de la CAN

attendus_CAN = 'BDD_attendus_janvier_2024'

In [ ]:
# ------------------ #
# IMPORT DES DONNÉES #
# ------------------ #

ENTREPOT_PATH = '~/Bureau/utils/data/' #chemin berenger equipe Agrosyst'
#ENTREPOT_PATH = '~/Bureau/utils/data/' #chemin berenger equipe Agrosyst
df = {}

def import_df(df_name, path_data, sep, index_col=None):
    df[df_name] = pd.read_csv(path_data+df_name+'.csv', sep = sep, index_col=index_col, low_memory=False).replace({'\r\n': '\n'}, regex=True)

def import_dfs(df_names, path_data, sep = ',', index_col=None, verbose=False):
    for df_name in tqdm(df_names) : 
        if(verbose) :
            print(" - ", df_name)
        import_df(df_name, path_data, sep, index_col=index_col)

tables_with_id = [
    'modele_decisionnel', 
    'modele_decisionnel_maitrise', 
    'modele_decisionnel_strategie',
    'sdc'
]

tables_without_id = [
    'modele_decisionnel_strategie_culture'
]

tables_CAN = [
    attendus_CAN
]

# import des données de l'entrepôt avec la colonne 'id' en index 
import_dfs(tables_with_id, ENTREPOT_PATH, sep = ',', index_col='id', verbose=False)

# import des données du magasin
import_dfs(tables_without_id, ENTREPOT_PATH, sep = ',', verbose=False)

# import des données CAN
import_dfs(tables_CAN, ENTREPOT_PATH, sep = ';', verbose=False)


100%|██████████| 1/1 [00:00<00:00,  7.15it/s]


### Constitution de modele_decisionnel_strategie_extanded
- ajout du nombre de cultures uniques

Une petite partie des décisionnels a des leviers renseignés sans culture associee. Une partie a aussi fleche un levier vers plusieurs cultures, ou plusieurs leviers vers une seule culture. La strategie consiste a remonter les id de culture avec leurs leviers jusqu'à l'échelle du sdc pour les compter à ce moment-la

In [409]:
# faire remonter l'information jusqu'au modele_decisionnel_strategie
left = df['modele_decisionnel_strategie']
right = df['modele_decisionnel_strategie_culture']
df['modele_decisionnel_strategie_extanded'] = pd.merge(left, right, left_index=True, right_on='modele_decisionnel_strategie_id', how='outer')

### Constitution de modele_decisionnel_maitrise_extanded
- ajout des leviers et cultures aux bioagresseurs

Besoin de prendre en compte 1/ le fait qu'il y a plusieurs leviers (modele_decisionnel_strategie_id) par bio agresseur (modele_decisionnel_maitrise_id) 2/ que pour certains bio agresseurs  (modele_decisionnel_maitrise_id) on n'a pas de leviers renseignés (modele_decisionnel_strategie_id)



In [410]:
# faire remonter l'information jusqu'au modele_decisionnel_maitrise
left = df['modele_decisionnel_maitrise']
right = df['modele_decisionnel_strategie_extanded']
df['modele_decisionnel_maitrise_extanded'] = pd.merge(left, right, left_index=True, right_on='modele_decisionnel_maitrise_id', how='outer')

### Constitution de modele_decisionnel_extanded

- ajout des generalites du modele decisionnel

on ajoute les donnees du modele decisionnel, notamment le type de MD "prevu" ou "constate"

In [411]:
# faire remonter l'information jusqu'au modele_decisionnel
left = df['modele_decisionnel']
right = df['modele_decisionnel_maitrise_extanded']
df['modele_decisionnel_extanded'] = pd.merge(left, right, left_index=True, right_on='modele_decisionnel_id', how='outer')

### Constitution de modele_decisionnel_sdc_extanded

- ajout des donnees du sdc

on ajoute les donnees necessaires du sdc : campagne, code sdc

In [412]:
# faire remonter l'information jusqu'au sdc
left = df['sdc'][['code_dephy', 'campagne' ]]
right = df['modele_decisionnel_extanded']
df['modele_decisionnel_sdc_extanded'] = pd.merge(left, right, left_index=True, right_on='sdc_id', how='outer')

### Constitution de controle_md, la base de controle des modeles decisionnels

Pour chaque code sdc, sdc_id, campagne : 
- ajout de la presence de md_constate
- ajout de la presence de md_prevu si absence de constate
- ajout du nombre de categorie d'objectif (tolerance) saisis par sdc dans les md "constate"
- ajout du nombre de cultures differentes saisies par sdc dans les md "constate"
- ajout du nombre moyen de leviers renseignes par culture par sdc dans les md "constate"
- ajout du nombre moyen de rubriques renseignees par culture par sdc dans les md "constate"
- ajout du nombre total de leviers renseignés en gestion des adventices par sdc dans les md "constate"
- ajout du nombre total de leviers renseignés en gestion des maladies par sdc dans les md "constate"
- ajout du nombre total de leviers renseignés en gestion des ravageurs par sdc dans les md "constate"

Puis mettre en forme en remplissant les NA par la valeur correspondante


In [413]:
# creer la BDD du controle et verifier la presence de constate, ou d'un prevu si pas de constate saisi. Mettre "NC" pour le prevu si un constate est saisi

df['controle_MD'] = df['modele_decisionnel_sdc_extanded'][['sdc_id', 'code_dephy', 'campagne']].drop_duplicates().dropna()
df['controle_MD']['MD_constate_saisi_obligatoire'] = np.where(df['controle_MD']['sdc_id'].isin(df['modele_decisionnel_sdc_extanded'].loc[df['modele_decisionnel_sdc_extanded']['categorie'] == 'OBSERVED','sdc_id']), 'OUI', 'NON')
df['controle_MD']['MD_prevu_saisi_hors_consignes'] = np.where(df['controle_MD']['sdc_id'].isin(df['modele_decisionnel_sdc_extanded'].loc[df['modele_decisionnel_sdc_extanded']['categorie'] == 'PLANNED','sdc_id']), 'OUI', 'NON')
df['controle_MD'].loc[df['controle_MD']['MD_constate_saisi_obligatoire'] == "OUI",'MD_prevu_saisi_hors_consignes'] = "NC"

#Nb de categorie d'objectif (tolerance) saisis par sdc dans les MD "constate" : on compte le nb de maitrise_id differents sous reserve que l'objectif soit non vide
left = df['controle_MD']
right = df['modele_decisionnel_sdc_extanded'].loc[(df['modele_decisionnel_sdc_extanded']['categorie'] == 'OBSERVED')&
                                                  (df['modele_decisionnel_sdc_extanded']['code_dephy'].notnull())&
                                                  (df['modele_decisionnel_sdc_extanded']['categorie_objectif'].notnull()) ,
                                                  ['sdc_id', 'modele_decisionnel_maitrise_id']
                                                    ].groupby(['sdc_id']).agg({'modele_decisionnel_maitrise_id': pd.Series.nunique
                                                                               }).rename(columns = {
                                                                                   'modele_decisionnel_maitrise_id':'Nb_total_categorie_objectif_renseignees'
                                                                                   })
df['controle_MD'] = pd.merge(left, right, left_on='sdc_id', right_on='sdc_id', how='outer')

#Nb de cultures saisies par sdc dans les MD "constate" : on compte le nb de culture_id differents par sdc_id
left = df['controle_MD']
right = df['modele_decisionnel_sdc_extanded'].loc[(df['modele_decisionnel_sdc_extanded']['categorie'] == 'OBSERVED')&
                                                  (df['modele_decisionnel_sdc_extanded']['code_dephy'].notnull())&
                                                  (df['modele_decisionnel_sdc_extanded']['culture_id'].notnull()) ,
                                                  ['sdc_id', 'culture_id']
                                                    ].groupby(['sdc_id']).agg({'culture_id': pd.Series.nunique
                                                                               }).rename(columns = {
                                                                                   'culture_id':'Nb_cultures_saisies'
                                                                                   })
df['controle_MD'] = pd.merge(left, right, left_on='sdc_id', right_on='sdc_id', how='outer')

#Nb moyen de leviers renseignés  par culture dans les MD "constate" : 
#on compte le nb de strategie_id (combinaison levier*bioagresseur) avec un levier non vide differents par culture_id, puis on moyenne à l'echelle du sdc
left = df['controle_MD']
right = df['modele_decisionnel_sdc_extanded'].loc[(df['modele_decisionnel_sdc_extanded']['categorie'] == 'OBSERVED')&
                                                  (df['modele_decisionnel_sdc_extanded']['code_dephy'].notnull())&
                                                  (df['modele_decisionnel_sdc_extanded']['levier'].notnull()) ,
                                                  ['sdc_id', 'culture_id','modele_decisionnel_strategie_id']
                                                    ].drop_duplicates().groupby(['sdc_id', 'culture_id']).count().groupby(['sdc_id']).mean().round(1).rename(columns = {
                                                                                   'modele_decisionnel_strategie_id':'Nb_moyen_leviers_par_culture'
                                                                                   })
df['controle_MD'] = pd.merge(left, right, left_on='sdc_id', right_on='sdc_id', how='outer')

#Nb moyen de rubriques (maitrise des adventices/maladies/ravageurs) renseignées  par culture dans les MD "constate" : 
#on compte le nb de strategie_id (combinaison levier*bioagresseur) avec un levier non vide differents par culture_id, puis on moyenne à l'echelle du sdc

left = df['controle_MD']
right = df['modele_decisionnel_sdc_extanded'].loc[(df['modele_decisionnel_sdc_extanded']['categorie'] == 'OBSERVED')&
                                                  (df['modele_decisionnel_sdc_extanded']['code_dephy'].notnull())&
                                                  (df['modele_decisionnel_sdc_extanded']['type_rubrique'].notnull()) ,
                                                  ['sdc_id', 'culture_id','type_rubrique']
                                                    ].drop_duplicates().groupby(['sdc_id', 'culture_id']).count().groupby(['sdc_id']).mean().round(1).rename(columns = {
                                                                                   'type_rubrique':'Nb_moyen_rubriques_par_culture'
                                                                                   })
df['controle_MD'] = pd.merge(left, right, left_on='sdc_id', right_on='sdc_id', how='outer')

#Nb total de leviers (md_strategie_id) renseignes par rubrique (adventice/maladie/ravageurs) dans les MD "constate" : 
#on compte le nb de strategie_id differents(combinaison levier*bioagresseur) avec un levier non vide par rubrique , puis on pivote chaque type de rubrique en colonne

left = df['controle_MD']
right = df['modele_decisionnel_sdc_extanded'].loc[(df['modele_decisionnel_sdc_extanded']['categorie'] == 'OBSERVED')&
                                                  (df['modele_decisionnel_sdc_extanded']['code_dephy'].notnull())&
                                                  (df['modele_decisionnel_sdc_extanded']['type_rubrique'].isin(['MALADIES','ADVENTICES','RAVAGEURS']))&
                                                  (df['modele_decisionnel_sdc_extanded']['levier'].notnull()) ,
                                                  ['sdc_id', 'modele_decisionnel_strategie_id','type_rubrique']
                                                    ].drop_duplicates().groupby(['sdc_id', 'type_rubrique'], as_index=False).agg({'modele_decisionnel_strategie_id': pd.Series.nunique
                                                                               }).pivot(index='sdc_id',
                                                                                        columns='type_rubrique',
                                                                                        values='modele_decisionnel_strategie_id'
                                                                                        ).fillna(0).rename(columns = {
                                                                                   'ADVENTICES' : 'Nb_total_leviers_adventices',
                                                                                   'RAVAGEURS' : 'Nb_total_leviers_ravageurs',
                                                                                   'MALADIES' : 'Nb_total_leviers_maladies'
                                                                                   })

df['controle_MD'] = pd.merge(left, right, left_on='sdc_id', right_on='sdc_id', how='outer')

# remplir les valeurs NA

df['controle_MD'].loc[df['controle_MD']['MD_constate_saisi_obligatoire']== "OUI"] = df['controle_MD'].loc[df['controle_MD']['MD_constate_saisi_obligatoire']== "OUI"].fillna(0)
df['controle_MD'].loc[df['controle_MD']['MD_constate_saisi_obligatoire']== "NON"] = df['controle_MD'].loc[df['controle_MD']['MD_constate_saisi_obligatoire']== "NON"].fillna("Abs")

#Ajuster le type de la campagne
df['controle_MD']['campagne'] = df['controle_MD']['campagne'].astype(int)


C:\Users\babiarm75n\AppData\Local\Temp\ipykernel_12880\1723717976.py:81: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['Abs' 'Abs' 'Abs' ... 'Abs' 'Abs' 'Abs']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df['controle_MD'].loc[df['controle_MD']['MD_constate_saisi_obligatoire']== "NON"] = df['controle_MD'].loc[df['controle_MD']['MD_constate_saisi_obligatoire']== "NON"].fillna("Abs")
C:\Users\babiarm75n\AppData\Local\Temp\ipykernel_12880\1723717976.py:81: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['Abs' 'Abs' 'Abs' ... 'Abs' 'Abs' 'Abs']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df['controle_MD'].loc[df['controle_MD']['MD_constate_saisi_obligatoire']== "NON"] = df['controle_MD'].loc[df['controle_MD']['MD_constate_saisi_oblig

### Constitution de controle_MD_attendus, la base de controle des modeles decisionnels par rapport aux attendus

A partir de la base de la CAN : 
- ne conserver que les donnees annuelles et PZ0 attendus pour chaque combinaison de code sdc * campagne
- y adjoindre la BDD controle_MD


In [ ]:

# On positionne la campagne en variable d'apres la BDD des attendus de la CAN. On en profite pour donner exactement les memes noms de variable que dans le fichier precedent
left = df[attendus_CAN].melt(id_vars= df[attendus_CAN].columns[:14],var_name='campagne'
                                     ).loc[ : , ['codes_SdC', 'codes_convention','campagne','value'] ].rename(columns = {
                                                                                   'value' : 'saisies_attendues',
                                                                                   'codes_SdC' : 'Numero_DEPHY',
                                                                                   'campagne' : 'Campagne'
                                                                                   })

right = df['controle_MD']

left['Campagne'] = pd.to_numeric(left['Campagne'])
right['campagne'] = pd.to_numeric(right['campagne'])

df['controle_MD_attendus'] = pd.merge(left, right, left_on=['Numero_DEPHY','Campagne'], right_on=['code_dephy','campagne'], how='left'
                                      ).drop(columns=['sdc_id','code_dephy','campagne'])



,Numero_DEPHY,codes_convention,Campagne,saisies_attendues,MD_constate_saisi_obligatoire,MD_prevu_saisi_hors_consignes,Nb_total_categorie_objectif_renseignees,Nb_cultures_saisies,Nb_moyen_leviers_par_culture,Nb_moyen_rubriques_par_culture,Nb_total_leviers_adventices,Nb_total_leviers_maladies,Nb_total_leviers_ravageurs
0,GCF35544,70CA79G2,2005,Pas de donnees attendues,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GCF26856,70CA79G2,2005,Pas de donnees attendues,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GCF35128,70CA79G2,2005,Pas de donnees attendues,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GCF35158,70CA79G2,2005,Pas de donnees attendues,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GCF28370,70CA79G2,2005,Pas de donnees attendues,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82796,GCF39334,70CI79PY,2024,Pas de donnees attendues,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82797,GCF39453,70CI79PY,2024,Pas de donnees attendues,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82798,GCF10056,87CA41GC,2024,Pas de donnees attendues,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82799,GCF10221,87CA41GC,2024,Pas de donnees attendues,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [415]:

#Ne conserver que les systemes qui doivent effectivement avoir un MD attendu : saisies annuelles et PZO

df['controle_MD_attendus'] = df['controle_MD_attendus'].loc[(df['controle_MD_attendus']['saisies_attendues'] == 'donnees annuelles attendues')  |
                                                             (df['controle_MD_attendus']['saisies_attendues'] == 'PZ0 attendu')  ]


#Gerer les NA : on indique qu'il n'y a pas de MD saisi

df['controle_MD_attendus']['MD_constate_saisi_obligatoire'] = df['controle_MD_attendus']['MD_constate_saisi_obligatoire'].fillna("NON")
df['controle_MD_attendus']['MD_prevu_saisi_hors_consignes'] = df['controle_MD_attendus']['MD_prevu_saisi_hors_consignes'].fillna("NON")
df['controle_MD_attendus'] = df['controle_MD_attendus'].fillna("Abs")



C:\Users\babiarm75n\AppData\Local\Temp\ipykernel_12880\2803484657.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['controle_MD_attendus']['MD_constate_saisi_obligatoire'] = df['controle_MD_attendus']['MD_constate_saisi_obligatoire'].fillna("NON")
C:\Users\babiarm75n\AppData\Local\Temp\ipykernel_12880\2803484657.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['controle_MD_attendus']['MD_prevu_saisi_hors_consignes'] = df['controle_MD_attendus']['MD_prevu_saisi_hors_consignes'].fillna("NON")


### Export de controle_MD et controle_MD_attendus

Export pour controle des saisies du reseau, uniquement pour les campagnes de controle


In [416]:
#Export des donnees
df['controle_MD'].drop(columns=['sdc_id']
                       ).loc[ (df['controle_MD']['campagne'] == campagne_controle ) |
                       (df['controle_MD']['campagne'] == campagne_controle -1 )
                       ].to_csv(ENTREPOT_PATH+'controle_MD.csv', index=False)


df['controle_MD_attendus'].loc[ (df['controle_MD_attendus']['Campagne'] == campagne_controle) |
                                (df['controle_MD_attendus']['Campagne'] == campagne_controle -1 )
                                ].to_csv(ENTREPOT_PATH+'controle_MD_attendus.csv', index=False)
